In [39]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import nltk
nltk.download('punkt_tab') # Garante que está disponível no ambiente virtual
from nltk.tokenize import word_tokenize




[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nanda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [4]:
from rank_bm25 import BM25Okapi


In [13]:
import pandas as pd

df = pd.read_csv("../data/processed/clean_data.csv")
df.head()

,NtsbNo,EventType,Mkey,EventDate,City,State,Country,ReportNo,N,HasSafetyRec,...,PurposeOfFlight,FAR,AirCraftDamage,WeatherCondition,Operator,ReportStatus,RepGenFlag,DocketUrl,DocketPublishDate,document
0,ERA24LA009,ACC,193248,2023-10-14T12:00:00Z,Myakka City,Florida,United States,NaN,N87621,False,...,PERS,91,Substantial,VMC,NaN,Completed,False,https://data.ntsb.gov/Docket?ProjectID=193248,11/16/2023 18:00,Date: 2023-10-14T12:00:00Z. Local: Myakka City...
1,ERA23LA379,ACC,193148,2023-09-26T16:51:00Z,Sanford,Florida,United States,NaN,N708WC,False,...,PERS,91,Substantial,VMC,NaN,Completed,False,https://data.ntsb.gov/Docket?ProjectID=193148,11/16/2023 18:00,"Date: 2023-09-26T16:51:00Z. Local: Sanford, Fl..."
2,CEN23LA394,ACC,192995,2023-09-02T06:54:00Z,Englewood,Colorado,United States,NaN,N20818,False,...,INST,91,Substantial,VMC,ATP USA,Completed,False,https://data.ntsb.gov/Docket?ProjectID=192995,10/17/2023 17:00,"Date: 2023-09-02T06:54:00Z. Local: Englewood, ..."
3,ERA23LA348,ACC,192935,2023-08-23T16:30:00Z,Pink Hill,North Carolina,United States,NaN,N8474J,False,...,PERS,91,Substantial,VMC,NaN,Completed,False,https://data.ntsb.gov/Docket?ProjectID=192935,12/7/2023 18:00,"Date: 2023-08-23T16:30:00Z. Local: Pink Hill, ..."
4,CEN23LA363,ACC,192876,2023-08-15T08:05:00Z,Lansing,Michigan,United States,NaN,N261SW,False,...,NaN,135,Substantial,IMC,"Ameriflight, LLC",Completed,False,https://data.ntsb.gov/Docket?ProjectID=192876,11/9/2023 18:00,"Date: 2023-08-15T08:05:00Z. Local: Lansing, Mi..."


In [ ]:
from nltk.tokenize import word_tokenize
from rank_bm25 import BM25Okapi

# Lista de documentos em texto
corpus = df["document"].tolist()

# Tokenização de cada documento
tokenized_corpus = [word_tokenize(doc.lower()) for doc in corpus]

# Instanciando o BM25
bm25 = BM25Okapi(tokenized_corpus)


In [38]:
from nltk.tokenize import word_tokenize
from rank_bm25 import BM25Okapi

def use_corpus(df):
    return [word_tokenize(doc.lower()) for doc in df["document"].tolist()]

def use_bm25(tokenized_corpus):
    return BM25Okapi(tokenized_corpus)


In [35]:
def buscar_documentos(pergunta, top_n=5):
    # Tokeniza a pergunta do usuário
    tokenized_query = word_tokenize(pergunta.lower())
    
    # Faz busca BM25
    scores = bm25.get_scores(tokenized_query)
    
    # Pega os top_n índices com maior score
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n]
    
    # Retorna os documentos e pontuações
    return [(df.iloc[i]["document"], scores[i]) for i in top_indices]


In [21]:
resultados = buscar_documentos("What caused the accident during takeoff?")
for i, (doc, score) in enumerate(resultados):
    print(f"\n--- Documento {i+1} (Score: {score:.2f}) ---")
    print(doc)



--- Documento 1 (Score: 6.60) ---
Date: 2023-06-26T11:00:00Z. Local: Elbert, Colorado, United States. Injuries: Minor (Fatais: 0, Serious: 0, Leves: 1). Probable Cause: The pilot's loss of directional control during takeoff and subsequent excessive use of brakes which resulted in a nose-over.  Contributing to the accident was his decision to takeoff downwind.. Condição climática: VMC. AirCraft Damage: Substantial. Propósito do voo: PERS. Operator: .

--- Documento 2 (Score: 4.33) ---
Date: 2023-05-30T14:05:00Z. Local: Santa Fe, New Mexico, United States. Injuries: Minor (Fatais: 0, Serious: 0, Leves: 1). Probable Cause: An encounter with a downdraft during landing that resulted in the pilot’s loss of control.As stated above, the initial PC was not incorrect, but it didn’t paint an accurate picture of what really happened.. Condição climática: VMC. AirCraft Damage: Unknown. Propósito do voo: PERS. Operator: .

--- Documento 3 (Score: 4.00) ---
Date: 2023-01-29T22:38:00Z. Local: Clevela

In [17]:
def recuperar_documentos(pergunta, bm25, documentos, k=3):
    """
    Retorna os k documentos mais relevantes com base na pergunta.
    """
    tokens_pergunta = word_tokenize(pergunta.lower())
    scores = bm25.get_scores(tokens_pergunta)
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k]
    
    return [documentos[i] for i in top_indices]


In [20]:
pergunta = "What caused the accident during takeoff?"
docs_relevantes = recuperar_documentos(pergunta, bm25, corpus, k=3)

for i, doc in enumerate(docs_relevantes):
    print(f"\nDocumento {i+1}:\n{doc}")



Documento 1:
Date: 2023-06-26T11:00:00Z. Local: Elbert, Colorado, United States. Injuries: Minor (Fatais: 0, Serious: 0, Leves: 1). Probable Cause: The pilot's loss of directional control during takeoff and subsequent excessive use of brakes which resulted in a nose-over.  Contributing to the accident was his decision to takeoff downwind.. Condição climática: VMC. AirCraft Damage: Substantial. Propósito do voo: PERS. Operator: .

Documento 2:
Date: 2023-05-30T14:05:00Z. Local: Santa Fe, New Mexico, United States. Injuries: Minor (Fatais: 0, Serious: 0, Leves: 1). Probable Cause: An encounter with a downdraft during landing that resulted in the pilot’s loss of control.As stated above, the initial PC was not incorrect, but it didn’t paint an accurate picture of what really happened.. Condição climática: VMC. AirCraft Damage: Unknown. Propósito do voo: PERS. Operator: .

Documento 3:
Date: 2023-01-29T22:38:00Z. Local: Cleveland, Texas, United States. Injuries: Minor (Fatais: 0, Serious: 